# SP500 Stock Demo — Notebook 06: Streamlit App (Snowflake)

- Minimal app: select ticker and model version
- Plot actual vs predicted returns
- Show latest PSI per feature


In [ ]:
# 0) Streamlit app code
import streamlit as st
import pandas as pd
from snowflake.snowpark.context import get_active_session

session = get_active_session()
session.sql("USE DATABASE SP500_STOCK_DEMO").collect()
session.sql("USE SCHEMA DATA").collect()

st.title('SP500 Forecast Demo')

# Model versions
models = session.sql("select name, versions from data.snowflake_ml_models where name = 'XGB_SP500_RET3M'").to_pandas()
versions = []
if not models.empty:
    import ast
    versions = ast.literal_eval(models.iloc[0]['VERSIONS'])
version = st.selectbox('Model version', options=versions if versions else ['V_1'])

# Ticker
symbols = session.table('SP500_TICKERS').select('TICKER').to_pandas()['TICKER'].tolist()
symbol = st.selectbox('Ticker', options=symbols[:200])

# Data fetch
preds = session.table('PREDICTIONS_SP500_RET3M').filter(f"TICKER = '{symbol}'").to_pandas()
features = session.table('PRICE_FEATURES').filter(f"TICKER = '{symbol}'").to_pandas()

df = preds.merge(features[['TICKER','TS','CLOSE']], on=['TICKER','TS'], how='left')
st.line_chart(df.set_index('TS')[['PREDICTED_RETURN']])

psi = session.table('DRIFT_PSI_SP500').to_pandas()
st.dataframe(psi)
